In [ ]:
import os, sys, pathlib, ipynbname, glob, shutil
import ansys.edb

from ansys.edb.core.database import Database

"""Import definition classes."""

from ansys.edb.core.definition.bondwire_def import (
    ApdBondwireDef,
    BondwireDef,
    BondwireDefType,
    Jedec4BondwireDef,
    Jedec5BondwireDef,
)
from ansys.edb.core.definition.component_def import ComponentDef
from ansys.edb.core.definition.component_model import (
    ComponentModel,
    DynamicLinkComponentModel,
    NPortComponentModel,
)
from ansys.edb.core.definition.component_pin import ComponentPin
from ansys.edb.core.definition.component_property import ComponentProperty
from ansys.edb.core.definition.dataset_def import DatasetDef
from ansys.edb.core.definition.debye_model import DebyeModel
from ansys.edb.core.definition.die_property import DieOrientation, DieProperty, DieType
from ansys.edb.core.definition.dielectric_material_model import (
    DielectricMaterialModel,
    DielectricMaterialModelType,
)
from ansys.edb.core.definition.djordjecvic_sarkar_model import DjordjecvicSarkarModel
from ansys.edb.core.definition.ic_component_property import ICComponentProperty
from ansys.edb.core.definition.io_component_property import IOComponentProperty
from ansys.edb.core.definition.material_def import MaterialDef, MaterialProperty, ThermalModifier
from ansys.edb.core.definition.material_property_thermal_modifier import (
    MaterialPropertyThermalModifier,
)
from ansys.edb.core.definition.multipole_debye_model import MultipoleDebyeModel
from ansys.edb.core.definition.package_def import PackageDef
from ansys.edb.core.definition.padstack_def import PadstackDef
from ansys.edb.core.definition.padstack_def_data import (
    PadGeometryType,
    PadstackDefData,
    PadstackHoleRange,
    PadType,
    SolderballPlacement,
    SolderballShape,
)
from ansys.edb.core.definition.port_property import PortProperty
from ansys.edb.core.definition.rlc_component_property import RLCComponentProperty
from ansys.edb.core.definition.solder_ball_property import SolderBallProperty
from ansys.edb.core.edb_defs import DefinitionObjType

"""Import geometry classes."""

from ansys.edb.core.geometry.arc_data import ArcData
from ansys.edb.core.geometry.point3d_data import Point3DData
from ansys.edb.core.geometry.point_data import PointData
from ansys.edb.core.geometry.polygon_data import ExtentType, PolygonData, PolygonSenseType
from ansys.edb.core.geometry.r_tree import RTree
from ansys.edb.core.geometry.triangle3d_data import Triangle3DData

"""Import hierarchy classes."""

from ansys.edb.core.hierarchy.cell_instance import CellInstance
from ansys.edb.core.hierarchy.component_group import ComponentGroup, ComponentType
from ansys.edb.core.hierarchy.group import Group
from ansys.edb.core.hierarchy.inst_array import InstArray
from ansys.edb.core.hierarchy.model import Model
from ansys.edb.core.hierarchy.netlist_model import NetlistModel
from ansys.edb.core.hierarchy.pin_group import PinGroup
from ansys.edb.core.hierarchy.pin_pair_model import PinPairModel
from ansys.edb.core.hierarchy.sparameter_model import SParameterModel
from ansys.edb.core.hierarchy.spice_model import SPICEModel
from ansys.edb.core.hierarchy.structure3d import MeshClosure, Structure3D
from ansys.edb.core.hierarchy.via_group import ViaGroup

"""Import layer classes."""

from ansys.edb.core.layer.layer import (
    DrawOverride,
    Layer,
    LayerType,
    LayerVisibility,
    TopBottomAssociation,
)
from ansys.edb.core.layer.layer_collection import (
    DielectricMergingMethod,
    LayerCollection,
    LayerCollectionMode,
    LayerTypeSet,
)
from ansys.edb.core.layer.stackup_layer import DCThicknessType, RoughnessRegion, StackupLayer
from ansys.edb.core.layer.via_layer import ViaLayer

"""Import layout classes."""

from ansys.edb.core.layout.cell import Cell, CellType, DesignMode
from ansys.edb.core.layout.layout import Layout
from ansys.edb.core.layout.mcad_model import McadModel
from ansys.edb.core.layout.voltage_regulator import PowerModule, VoltageRegulator

"""Import layout instance classes."""

from ansys.edb.core.layout_instance.layout_instance import LayoutInstance
from ansys.edb.core.layout_instance.layout_instance_context import LayoutInstanceContext
from ansys.edb.core.layout_instance.layout_obj_instance import LayoutObjInstance
from ansys.edb.core.layout_instance.layout_obj_instance_2d_geometry import (
    LayoutObjInstance2DGeometry,
)
from ansys.edb.core.layout_instance.layout_obj_instance_3d_geometry import (
    LayoutObjInstance3DGeometry,
)
from ansys.edb.core.layout_instance.layout_obj_instance_geometry import LayoutObjInstanceGeometry

"""Import net classes."""

from ansys.edb.core.net.differential_pair import DifferentialPair
from ansys.edb.core.net.extended_net import ExtendedNet
from ansys.edb.core.net.net import Net
from ansys.edb.core.net.net_class import NetClass

"""Import primitive classes."""

from ansys.edb.core.primitive.primitive import (
    BackDrillType,
    BoardBendDef,
    Bondwire,
    BondwireCrossSectionType,
    BondwireType,
    Circle,
    PadstackInstance,
    Path,
    PathCornerType,
    PathEndCapType,
    Polygon,
    Primitive,
    PrimitiveType,
    Rectangle,
    RectangleRepresentationType,
    Text,
)

"""Import simulation setup classes."""

from ansys.edb.core.simulation_setup.adaptive_solutions import (
    AdaptiveFrequency,
    BroadbandAdaptiveSolution,
    MatrixConvergenceData,
    MatrixConvergenceDataEntry,
    MultiFrequencyAdaptiveSolution,
    SingleFrequencyAdaptiveSolution,
)
from ansys.edb.core.simulation_setup.hfss_simulation_settings import (
    AdaptType,
    BasisFunctionOrder,
    HFSSAdvancedMeshingSettings,
    HFSSAdvancedSettings,
    HFSSDCRSettings,
    HFSSGeneralSettings,
    HFSSSettingsOptions,
    HFSSSimulationSettings,
    HFSSSolverSettings,
    SolverType,
)
from ansys.edb.core.simulation_setup.hfss_simulation_setup import HfssSimulationSetup
from ansys.edb.core.simulation_setup.mesh_operation import (
    LengthMeshOperation,
    MeshOperation,
    SkinDepthMeshOperation,
)
from ansys.edb.core.simulation_setup.raptor_x_simulation_settings import (
    RaptorXAdvancedSettings,
    RaptorXGeneralSettings,
    RaptorXSimulationSettings,
)
from ansys.edb.core.simulation_setup.raptor_x_simulation_setup import RaptorXSimulationSetup
from ansys.edb.core.simulation_setup.simulation_settings import (
    AdvancedMeshingSettings,
    AdvancedSettings,
    SettingsOptions,
    SimulationSettings,
    SolverSettings,
    ViaStyle,
)
from ansys.edb.core.simulation_setup.simulation_setup import (
    FreqSweepType,
    HFSSRegionComputeResource,
    InterpolatingSweepData,
    SimulationSetup,
    SimulationSetupType,
    SweepData,
)
from ansys.edb.core.simulation_setup.siwave_dcir_simulation_settings import (
    SIWaveDCIRSimulationSettings,
)
from ansys.edb.core.simulation_setup.siwave_dcir_simulation_setup import SIWaveDCIRSimulationSetup
from ansys.edb.core.simulation_setup.siwave_simulation_settings import (
    SIWaveAdvancedSettings,
    SIWaveDCAdvancedSettings,
    SIWaveDCSettings,
    SIWaveGeneralSettings,
    SIWaveSimulationSettings,
    SIWaveSParameterSettings,
    SParamDCBehavior,
    SParamExtrapolation,
    SParamInterpolation,
)
from ansys.edb.core.simulation_setup.siwave_simulation_setup import SIWaveSimulationSetup

"""Import terminal classes."""

from ansys.edb.core.terminal.terminals import (
    BoundaryType,
    BundleTerminal,
    Edge,
    EdgeTerminal,
    EdgeType,
    HfssPIType,
    PadEdge,
    PadstackInstanceTerminal,
    PinGroupTerminal,
    PointTerminal,
    PrimitiveEdge,
    SourceTermToGroundType,
    Terminal,
    TerminalInstance,
    TerminalInstanceTerminal,
    TerminalType,
)

"""Import utility classes."""

from ansys.edb.core.utility.heat_sink import HeatSink
from ansys.edb.core.utility.hfss_extent_info import HfssExtentInfo
from ansys.edb.core.utility.material_property_thermal_modifier_params import (
    AdvancedQuadraticParams,
    BasicQuadraticParams,
)
from ansys.edb.core.utility.port_post_processing_prop import PortPostProcessingProp
from ansys.edb.core.utility.rlc import PinPair, PinPairRlc, Rlc
from ansys.edb.core.utility.temperature_settings import TemperatureSettings
from ansys.edb.core.utility.transform3d import Transform3D
from ansys.edb.core.utility.transform import Transform
from ansys.edb.core.utility.value import Value

from ansys.edb.core.session import launch_session

In [ ]:
def copy_input():
    if os.path.exists(get_output_filename()):
        shutil.rmtree(get_output_filename())
    shutil.copytree(get_input_filename(), get_output_filename())
    return get_output_filename()

def get_fixture_file(filename):
    example_name = ipynbname.name()
    return os.path.join(os.getcwd(), 'fixtures', example_name, filename)
    
def get_input_filename():
    example_name = ipynbname.name()
    return os.path.join(os.getcwd(), f'{example_name}.aedb')
    
def get_output_filename():
    example_name = ipynbname.name()
    return os.path.join(get_output_dir(), f'{example_name}.aedb')

def get_output_files():
    example_name = ipynbname.name()
    return os.path.join(get_output_dir(), f'{example_name}.*')

def get_output_dir():
    return os.path.join(os.getcwd(), "results")

In [ ]:
# this will wipe away existing db/session if user runs this script multiple times in same jupyter kernel session.
%run helpers/example_teardown.ipynb

if 'ANSYSEM_EDB_EXE_DIR' in os.environ:
    EXE_DIR = os.environ['ANSYSEM_EDB_EXE_DIR']
else:
    EXE_DIR = None

session = launch_session(EXE_DIR, 50051)

In [ ]:
if not pathlib.Path(get_output_dir()).is_dir():
    os.makedirs(get_output_dir())

for f in glob.glob(get_output_files()):
    if pathlib.Path(f).is_dir():
        shutil.rmtree(f)
    else:
        os.remove(f)

if pathlib.Path(get_input_filename()).is_dir():
    db = Database.open(copy_input(), False)
    cell = db.circuit_cells[0]
    layout = cell.layout
    print('db, cell, layout variables set to the default from edb design.')
else:
    db = Database.create(get_output_filename())
    cell = Cell.create(db, CellType.CIRCUIT_CELL, "Example")
    layout = cell.layout
    gnd_net = Net.create(layout, 'GND')
    print('clean design created with default db, cell, layout, gnd_net variables.')